# 10.1 Vector : 사용자 정의 시퀀스형
- 구성을 이용해서 벡터 구현
- 요소들을 실수형 배열에 저장하기 위한 필요한 메서드들 구현
- 벡터가 불변 균일 시퀀스처럼 작동하게 만들기 위한 필요한 메서드들 구현

# 10.2 Vector 버전 #1 : Vector2d 호환
- 시퀀스 생성자는 내장 시퀀스처럼 반복형을 인수로 받게 만드는 것이 좋다.
- repr()이 생성한 문자열은 문자열은 생략 기호 (...)로 축약되었다. 이와 같이 제한된 길이로 표현하려면 reprlib 모듈 사용할 것

In [1]:
# vector2d_v3.py

from array import array
import math

class Vector2d:
    typecode = 'd' # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y) # 미리 실수로 변환하는 센스
        
    @property # 게터 메서드를 나타냄
    def x(self):
        return self.__x        
        
    @property
    def y(self):
        return self.__y

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)        
    
    def __iter__(self): # 이걸 구현하면 x,y = my_vector 처럼 쓸 수 있다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])+
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '<{}, {}>'            
        components = (format(c, fmt_spec) for c in coords)
        return '({}, {})'.format(*components)

    @classmethod # 클래스 메서드
    def frombytes(cls, octets): # self 매개변수가 없고 대신 자신이 cls로 전달됨
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

# 10.3 프로토콜과 덕 타이핑
- 객체 지향 프로그래밍에서 프로그램에서 프로토콜은 문서에만 정의되어 있고 실제 코드에서는 정의되지 않는 비공식 인터페이스다.
    - e.g.) 파이썬의 시퀀스 프로토콜은 \_\_len\_\_() 과 \_\_getitem\_\_() 메서드를 동반할 뿐이다.
    - 표현 시그너처와 의미에 따라 이 메서드들을 구현하는 어떠한 클래스도 시퀀스가 필요한 곳에 사용할 수 있다.
    - 그 클래스의 슈퍼 클래스가 무엇인지 중요하지 않으며, 단지 필요한 메서드만 제공하면 된다.

- 예제 10.3 코드 어디에서도 시퀀스 프로토콜을 따른다고 정의한 곳은 없다.
- 이 클래스가 시퀀스처럼 동작하기 때문에 시퀀스인 것이다.
- 프로토콜이 비공식적이며 강제로 적용되는 사항이 아니므로 클래스가 사용되는 특정 환경에 따라 프로토콜의 일부만 구현할 수 있다.
    - 예를 들어 반복을 지원하려면 \_\_getitem\_\_() 메서드만 구현하면 되며, \_\_len\_\_() 메서드를 구현할 필요는 없다.

```python
# 예제 10.3
import collections

Card  = collections.namedtuple("Card",['rank','suit'])

class FrenchDeck:
    ranks = [ str(n) for n in range(2,11) ] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank,suit) for suit in self.suits for rank in self.ranks ]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
```

# 10.4 Vector 버전 #2: 슬라이스 가능한 시퀀스
- self.\_components array 객체처럼, 객체에 sequence attribute 를 위임할 수 있다면 sequence protocol 구현(supporting)에 용이하다. 예로서 \_\_len\_\_ 과 \_\_getitem\_\_ 이 있다.
```python
class Vector:
    # 중략
    # ...
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        return self._components[index]
```

## 10.4.1 슬라이싱의 작동 방식
- Vector 를 슬라이싱해서 Vector 객체를 생성하려면, 슬라이싱 연산을 배열에 위임하면 안되며, \_\_getitem\_\_() 메서드가 받은 인수를 분석해서 제대로 처리해야한다.
- slice 는 내장된 자료형이며, slice 객체를 조사하면 start, stop, step 속성과 indices() 메서드를 볼 수 있다.
    - indices() 메서드는 주어진 길이의 시퀀스 경계 안에 들어가도록 조정된 0이나 양수인 start, stop, stride 로 구성된 '정규화된' 튜플을 생성한다. 우리가 구현할 Vector 코드에서는 slice 인수를 받을 때 \_components 배열에 처리를 위임할 것이므로 slice.indices() 메서드 구현할 필요가 없지만, 기반 시퀀스가 제공하는 서비스에 의존할 수 없을 때 이 메서드가 큰 도움이 된다. 
```python
# 'ABCDE' 처럼 길이가 5인 시퀀스에 적용한 슬라이스의 예는 다음과 같다.
slice(None,10,2).indices(5)
(0,5,2)
slice(-3,None,None).indices(5)
(2,5,1)
```

## 10.4.2 슬라이스를 인식하는  \_\_getitem\_\_()
```python
def __len__(self):
    return len(self._components)

def __getitem__(self,index):
    cls = type(self)
    if isinstance(index, slice):
        return cls(self._components[index])
    elif isinstance(index, numbers.Integral):
        return self._components[index]
    else:
        msg = '{cls.__name__} indices must be integers'
        raise TypeError(msg.format(cls=cls))

```

# 10.5 Vector 버전 #3: 동적 속성 접근
- 앞에 있는 요소 몇 개는 v[0], v[1], v[2] 대신 x,y,z 로 접근할 수 있으면 편리할 것임. \_\_getattr\_\_() 특별 메서드를 이용하면 깔끔하게 구현 가능
- 속성을 찾지 못하면 인터프리터는 \_\_getattr\_\_() 메서드를 호출한다. myobj.x 표현식이 주어지면, 파이썬은 myobj 객체에 x속성이 있는지 검사하고, 속성이 없으면 이 객체의 클래스(myobj.\_\_class\_\_) 에서 더 검색한다. 그러고 나서 상속 그래프를 따라 계속 올라간다. 그래도 x 속성을 찾지 못하면 self 와 속성명을 문자열 ( 예를 들어 'x') 로 전달해서 myobj 클래스에 정의된 \_\_getattr\_\_() 메서드를 호출한다.

```python
shortcut_names = 'xyzt'

def __getattr__(self, name):
    cls = type(self)
    if len(name) ==1:
        pos = cls.shortcut_names.find(name)
        if 0 <= pos < len(self._components):
            return self._components[pos]
    msg = '{.__name__!r} object has no attribute {!r}'
    raise AttributeError(msg.format(cls, name))

```

- 그러나 getattr 만 구현시 368p 같은 불일치 문제가 발생할 수 있다. ( v.x 가 정의되면 x 를 찾을 때 \_\_getattr\_\_()메서드가 호출되지 않으며 \_\_getattr\_\_() 메서드는 self.\_components 이외의 다른 속성에는 주의를 기울이지 않기 때문에 발생 ) 
- 이와 같은 불일치 문제를 해결하기 위해 \_\_setattr\_\_() 메서드를 구현해야한다.
- 객체 동작의 불일치를 피하려면 \_\_getattr\_\_() 을 구현할 때 \_\_setattr\_\_() 도 함께 구현해야한다.
- \_\_getattr\_\_() works when you want to lookup an attribute that dosen't exist in this class
- \_\_setattr\_\_() works when you assign a value to an attribute that dosen't exist in this class
```python
def __setattr__(self, name, value):
    cls = type(self)
    if len(name)== 1:
        if name in cls.shortcut_names:
            error = 'readonly attribute {attr_name!r}'
        elif name.islower():
            error = "can't set attributes 'a' to 'z' in {cls_name!r}"
        else:
            error = ''
        if error:
            msg = error.format(cls_name = cls.__name__, attr_name = name)
            raise AttributeError(msg)
    super().__setattr__(name, value)
```

# 10.6 Vector 버전 #4: 해싱 및 더 빠른 ==
- 모든 벡터 요소의 헤시를 계싼하는 연산은 reduce()에 딱 맞는다. 
- reduce() 의 핵심은 일련의 값을 하나로 줄이는 것이다.
    - reduce()가 받는 첫 번째 인수는 두 개의 인수를 받는 함수, 두 번째 인수는 반복형 객체다.
- operator 모듈은 모든 파이썬 중위 연산자를 함수 형태로 제공해서 람다를 사용할 필요성을 줄여준다. 

In [3]:
# 1 st method
n = 0 
for i in range(6):
    n ^= i
    
print(n)

1


In [4]:
# 2nd method
import functools
functools.reduce( lambda a,b : a^b, range(6))

1

In [5]:
# 3th method
import operator
functools.reduce( operator.xor, range(6))

1

```python

from array import array
import reprlib
import math
import functools
import operator

class Vector:
    typecode = 'd'
    
    # 중간 코드 생략
    
    # __eq__() 와 __hash__() 가 밀접히 작동해야 하므로, 소스 코드 안에서 가까이 두는 습관을 들이는 것이 좋다
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)

```

```python
# 제너레이터 표현식 대신 맵을 사용하면 맵 단계가 훨씬 더 잘 드러난다.

def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes)
```

```python
# 수천 개의 요소를 가질 수 있는 Vector 객체의 경우 피연산자 전체를 복사해서 튜플 두 개를 만드는 것은 매우 비효율적임
# Vector 객체를 다른 객체나 반복형과 비교할 때는 예제 10-13 같이 구현하는 것이 좋다.

# 예제 13-10
def __eq__(self, other):
    if len(self) != len(other):
        return False
    for a,b in zip(self, other):
        if a != b:
            return False
    return True
```

```python
# all()함수를 사용하면 for 루프와 동일한 계산을 단 한 줄에 할 수 있다. 해당 요소 간의 비교가 모두 True면, 결과도 True 다.
# 비교하는 도중에 다른 요소가 나오면, 즉 비교가 False 면 all()은 바로 False를 반환한다. all()함수를 이용해서 구현한 __eq__()메서드는 예제 10-14와 같다.

def __eq__(self, other):
    return len(self) == len(other) and all(a==b for a,b in zip(self,other))
```

# 10.7 Vector 버전 #5: 포매팅
- 별 내용 없음

In [6]:
# vector_v5.py

from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools # chain() 함수를 사용하기 위함

class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    
    def __init__(self, components):
        # 벡터 요소를 배열로 저장
        self._components = array(self.typecode, components)
    
    """시퀀스 프로토콜 구현"""
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self) # 객체의 클래스를 가져옴
        if isinstance(index, slice): # index가 슬라이스이면
            return cls(self._components[index]) # Vector 객체를 생성
        elif isinstance(index, numbers.Integral): # index가 정수형이면
            return self._components[index] # 해당 항목 반환
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls)) 
    """시퀀스 프로토콜 구현 종료"""
    
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1: # name이 한글자이면 
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components): # 포지션이 범위 내에 있으면 배열 항목 반환
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}' 
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}' # xyzt 중 하나는 구체적으로 오류 발생
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}" # 그외 소문자면 일반적 메세지 오류 발생
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        # 에러가 발생하지 않을 때는 정상적으로 __setattr__() 메서드 호출
        super().__setattr__(name, value) 
        
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1]<0):
            return math.pi * 2 - a
        else:
            return a
        
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'): # 초구면좌표
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                    self.angles()) 
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))
    
    def __iter__(self):
        return iter(self._components) # 반복할 수 있도록 구현
    
    def __repr__(self):
        components = reprlib.repr(self._components) # 제한된 길이로 출력
        components = components[components.find('['):-1] # 문자열 중 앞에 나오는 "array('d'," 를 제거
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(self._components))
    
    def __eq__(self, other):
        if len(self) != len(other): # 길이가 다르면 False
            return False
        for a, b in zip(self, other): # 제너레이터로부터 하나씩 비교
            if a != b:
                return False
        return True
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components) # 제너레이터 표현식 이용
        return functools.reduce(operator.xor, hasehs, 0) # 초기값을 0으로 함
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # 언패킹할 필요가 없음